In [1]:
import numpy as np
import pygsti as pig
import matplotlib.pyplot as plt
import random
import time
import pandas
from scipy.linalg import expm, sqrtm
# from qutip import *
from tqdm import tqdm

from pygsti.modelmembers.operations import LindbladErrorgen as _LinbladErrorgen
from pygsti.modelmembers.operations import EmbeddedErrorgen as _EmbeddedErrorgen
from pygsti.modelmembers.operations import ExpErrorgenOp as _ExpErrorgenOp
from pygsti.modelmembers.operations import ComposedOp as _ComposedOp
import  pygsti.modelmembers as _mdmb
import pygsti.processors as _proc

from pygsti.tools import pdftools as _pdftools

from pygsti.circuits.gstcircuits import create_lsgst_circuits

from pygsti.models.memberdict import OrderedMemberDict as _OrderedMemberDict
from pygsti.baseobjs.label import Label as _Label, CircuitLabel as _CircuitLabel
from pygsti.circuits import Circuit
import pygsti.circuits as _circ
import pygsti.models as _mdl
from pygsti.modelmembers import operations as _op

In [2]:
def random_circuit(depth, proc_spec):
    """
    works for any 1-qubit model pack and the XYZICnot 2-qubit pack
    """
    layers = []
    for i in range(depth):
        gate = random.choice(proc_spec.gate_names)
        if gate == '(idle)':
            pass
        elif gate == 'Gcnot':
            layers.append((gate, 0, 1))
        else:
            layers.append((gate, random.choice(proc_spec.qubit_labels)))
    return Circuit(layers, proc_spec.qubit_labels)

In [3]:
def vector_from_outcomes(outcomes, num_outcomes):
    vecout = np.zeros((num_outcomes))
    for key in outcomes.keys():
        vecout[int(key[0], 2)] = outcomes[key]
    return(vecout)

def matrix_from_jacob(jacob, num_outcomes):
    matout = np.zeros((num_outcomes, len(jacob['0'*int(np.log2(num_outcomes))])))
    for key in jacob.keys():
        matout[int(key[0], 2), :] = np.array(jacob[key])
    return matout

def tensor_from_hessian(hessian, hilbert_dims):
    """
    returns a 3d-array that when dotted into the state returns the jacobian 
    """
    num_params = len(hessian['0'*int(np.log2(hilbert_dims))])
    tensor_out = np.zeros((hilbert_dims, num_params, num_params))
    for key in hessian.keys():
        tensor_out[int(key[0], 2), :, :] = hessian[key]
    return tensor_out

In [4]:
def compare_models(true_model, model1, model2, circuit_list, name1='x', name2='y'):
    distribution1 = []
    distribution2 = []
    mx = 0
    for idx, circ in enumerate(circuit_list): 
        hilbert_dims = 2**circ.width
        true_outcomes = true_model.probabilities(circ)
        outcome_set1 = model1.probabilities(circ)
        outcome_set2 = model2.probabilities(circ)
        dist1 = _pdftools.tvd(true_outcomes, outcome_set1)
        dist2 = _pdftools.tvd(true_outcomes, outcome_set2)
        if dist1 > mx:
            mx = dist1
        if dist2 > mx: 
            mx = dist2
        vtrue = vector_from_outcomes(true_outcomes, hilbert_dims)
        v1 = vector_from_outcomes(outcome_set1, hilbert_dims)
        v2 = vector_from_outcomes(outcome_set2, hilbert_dims)
        distribution1.append(vtrue - v1)
        distribution2.append(vtrue - v2)
        plt.scatter(dist1, dist2)
    plt.xlabel(name1)
    plt.ylabel(name2)
    plt.plot((0, mx), (0, mx), c='black')
    plt.show()
    return (distribution1, distribution2)

In [5]:
def calculate_jacobs(circ_list, model):
    jacobs = {}
    for circ in circ_list:
        hilbert_dims = 2**circ.width
        jacobs[circ] = matrix_from_jacob(model.sim.dprobs(circ), hilbert_dims)
    return jacobs

In [6]:
from pygsti.processors import CliffordCompilationRules as CCR

def make_rb_param(noise_model, model_pack, length_powers=5, circuits_at_length=10):
    pspec = model_pack.processor_spec()
    
    depths = [2**i for i in range(length_powers)]
    
    compilations = {'absolute': CCR.create_standard(pspec, 'absolute', ('paulis', '1Qcliffords'), verbosity=0),            
                'paulieq': CCR.create_standard(pspec, 'paulieq', ('1Qcliffords', 'allcnots'), verbosity=0)}

    design = pig.protocols.DirectRBDesign(model_pack.processor_spec(), compilations, depths, circuits_at_length, qubit_labels=model_pack.processor_spec().qubit_labels, sampler='edgegrab', 
                                           samplerargs=[0.5], randomizeout=True,
                                           citerations=20)
    pig.io.write_empty_protocol_data(design, 'RB_Data', clobber_ok=True)
    pig.io.fill_in_empty_dataset_with_fake_data(noise_model, 'RB_Data/data/dataset.txt', num_samples=1000)
    data = pig.io.load_data_from_dir('RB_Data')
    protocol = pig.protocols.RB()
    results = protocol.run(data)
    return (results.fits['full'].estimates['r'], data)

In [7]:
def make_dirichlet_covar(counts, hilbert_dims):
    total_counts = sum([counts[key] for key in counts.keys()])    
    count_vec = np.ones(hilbert_dims)
    for key in counts.keys():
        count_vec[int(key[0], 2)] += counts[key]
    prefactor = 1/( (total_counts + hilbert_dims)**2 * (total_counts + hilbert_dims + 1) )
    meas_covar = prefactor*(
        (total_counts + hilbert_dims)*np.diag(count_vec) - np.outer(count_vec, count_vec)
    )
    return meas_covar

In [8]:
def make_multinom_covar(prob_vec):
    return np.eye(len(prob_vec)) - np.outer(prob_vec, prob_vec)

In [9]:
from scipy.linalg import block_diag

class ExtendedKalmanFilter():
    def __init__(self, model, P0):
        self.model = model
        self.P = P0
        
    def update(self, circ_list, data_set, jdict, hdict, stab_noise=None, max_itr=1, itr_eps=1e-4):
        
        
        prior_covar = self.P
        prior_state = self.model.to_vector()
        
        for itr in range(max_itr):
            Smat = np.zeros((0,0))
            total_innov = np.zeros((0))
            total_jac = np.zeros((0,len(self.model.to_vector())))
        
            for circ in circ_list:
                counts = data_set[circ].counts
                total_counts = sum([counts[key] for key in counts.keys()])    
                if total_counts > num_samples:
                    print('counts exceeded num_samples, ', total_counts)
                hilbert_dims = 2**(circ.width)
                prior_state = self.model.to_vector()

                # 0) find the mean estimate for the circuit outcome under the self.model
                p_model = np.ones(hilbert_dims)
                probs = self.model.probabilities(circ)
                for key in probs.keys():
                    p_model[int(key[0], 2)] += total_counts*probs[key]
                p_model = (1/(total_counts+hilbert_dims))*p_model

                # calculate jacobian
                jacob = jdict[circ] + hdict[circ]@prior_state

                # 1) calculate your observation frequencies
                observation = np.ones(hilbert_dims)
                for key in counts.keys():
                    observation[int(key[0], 2)] += counts[key]
                observation = (1/(total_counts+hilbert_dims))*observation

                # 2) calculate the covaraiance of the observation and add model noise
                meas_covar = (1/(total_counts+hilbert_dims))*make_multinom_covar(observation)
                if stab_noise is not None:
                    meas_covar += stab_noise
                    
                # 3) Kalman gain
                smat = np.linalg.pinv(jacob@prior_covar@jacob.T + meas_covar, 1e-6)
                
                innovation = observation - p_model

                Smat = block_diag(Smat, smat)
                total_innov = np.hstack([total_innov, innovation])
                total_jac = np.vstack([total_jac, jacob])
            Kgain = prior_covar@total_jac.T@Smat
            post_state = prior_state + Kgain@total_innov
            if np.linalg.norm(post_state - prior_state) < itr_eps:
                break
            prior_state = post_state
        self.P = prior_covar - Kgain@total_jac@prior_covar
        self.model.from_vector(post_state)

In [10]:
def regression_fit(circ_list, data_set, ref_model, jdict):
    num_params = len(ref_model.to_vector())
    pvec = np.zeros((0))
    jmat = np.zeros((0, num_params))
    
    for circ in circ_list:
        hilbert_dims = 2**circ.width
        
        counts = vector_from_outcomes(data_set[circ].counts, hilbert_dims)
        total_counts = sum(counts)
        observation = (counts + np.ones((hilbert_dims)))/(total_counts + hilbert_dims)
        
        p_model = np.ones((hilbert_dims))
        probs = ref_model.probabilities(circ)
        for key in probs.keys():
            p_model[int(key[0], 2)] += total_counts*probs[key]
        p_model = p_model/(total_counts + hilbert_dims)
        
        pvec = np.hstack((pvec, observation-p_model))
        jmat = np.vstack((jmat, jdict[circ]))
    return np.linalg.pinv(jmat, 1e-4)@pvec

In [11]:
from filterpy.kalman import KalmanFilter

class LinearKalmanFilter(KalmanFilter):
    def __init__(self, x0, P0, hilbert_dims):
        super().__init__(len(x0), hilbert_dims)
        self.x = x0
        self.P = P0
        
        self.F = np.eye(len(x0))
        self.Q = np.zeros((len(x0), len(x0)))
        
    def update_filter(self, circ, count_vec, pvec_model, jacob, stab_noise=None):
        
        total_counts = sum(count_vec)
        
        hilbert_dims = 2**circ.width
        
        prediction = (total_counts*pvec_model + np.ones(hilbert_dims))/(total_counts + hilbert_dims)
        observation = (count_vec + np.ones(hilbert_dims))/(total_counts + hilbert_dims)
        
        self.H = jacob
        shot_noise = (1/total_counts)*make_multinom_covar(observation)
        if stab_noise is not None:
            shot_noise += stab_noise
        self.R = shot_noise
        
        self.update(observation - prediction)

In [12]:
def compare_model_list(noise_model, model_list, circ_list):
    # report average and std-dev of tvd errors 
    tvds = np.zeros((len(circ_list), len(model_list)))
    print('Total number of circuits: ', len(circ_list))
    for idx, circ in tqdm(enumerate(circ_list)):
        for i, model in enumerate(model_list):
            tvds[idx, i] = _pdftools.tvd(model.probabilities(circ), noise_model.probabilities(circ))
    avg_tvds = np.sum(tvds, axis=0)/len(tvds)
    var_tvds = np.zeros(avg_tvds.shape)
    for idx, tvd in enumerate(tvds):
        var_tvds += (tvd - avg_tvds)**2/len(tvds)
    sigma_tvds = np.sqrt(var_tvds)
    return (avg_tvds, sigma_tvds)

In [13]:
import pygsti
from pygsti.report import reportables as rptbl
basis = pygsti.baseobjs.Basis.cast("pp",4) # 1-qubit Pauli basis (2x2 matrices)

def avg_eigenvalue_infidelity(model1, model2, gateset):
    aei = 0
    for element in gateset:
        aei += rptbl.eigenvalue_entanglement_infidelity(model1[element].to_dense(), model2[element].to_dense(), basis)
    return aei/len(gateset)

# [1] Pick a Model

In [54]:
from pygsti.modelpacks import smq1Q_XYI as _smq1Q_XYI
from pygsti.modelpacks import smq1Q_XYZI as _smq1Q_XYZI
from pygsti.modelpacks import smq1Q_XYI as _smq1Q_XZ
from pygsti.modelpacks import smq2Q_XYZICNOT as _smq2Q_XYZICNOT
MODEL_PACK = _smq1Q_XYZI
REF_MODEL = MODEL_PACK.target_model('H+S')

In [15]:
GATES = [('Gzpi2', 0), ('Gypi2', 0), ('Gxpi2', 0)]

# [2] Design Experiments and Calculate the Jacobians and Hessians

In [16]:
# make GST design
import pygsti

target_model = MODEL_PACK.target_model('H+S')      # a Model object
prep_fiducials = MODEL_PACK.prep_fiducials()  # a list of Circuit objects
meas_fiducials = MODEL_PACK.meas_fiducials()  # a list of Circuit objects
germs = MODEL_PACK.germs()                    # a list of Circuit objects
maxLengths = [1,2, 4, 8, 16, 32]  
exp_design = pygsti.protocols.StandardGSTDesign(target_model, prep_fiducials, meas_fiducials,
                                                germs, maxLengths)
gst_circuits = exp_design.all_circuits_needing_data

In [17]:
jdict_gst = dict()
hdict_gst = dict()
print("total circuits: ", len(gst_circuits))
for idx, circ in tqdm(enumerate(gst_circuits)):
    jdict_gst[circ] = matrix_from_jacob(REF_MODEL.sim.dprobs(circ), 2**circ.width)
    hdict_gst[circ] = tensor_from_hessian(REF_MODEL.sim.hprobs(circ), 2**circ.width)

total circuits:  1570


1570it [17:33,  1.49it/s]


# [3] Add noise and make the filter model

In [128]:
# make a model from a model pack
filter_model = REF_MODEL.copy()
regression_model = REF_MODEL.copy()
noise_model = MODEL_PACK.target_model()
noise_model = noise_model.depolarize(max_op_noise=0.0007, max_spam_noise=0.0001)
noise_model = noise_model.rotate(max_rotate=0.0007)

In [129]:
# preint average entanglement infidelity of the noise model
aei = 0
for op in list(REF_MODEL.operations.keys()):
    aei += 1-rptbl.entanglement_fidelity(noise_model[op], REF_MODEL[op].to_dense(), basis)
print(aei)

0.0008420683071486135


# [4] Run an RB experiment and collect outcome data for the filter run

In [130]:
rb_param, data = make_rb_param(noise_model, MODEL_PACK)
print(rb_param)

- Sampling 10 circuits at DRB length 1 (1 of 5 depths) with seed 551104
- Sampling 10 circuits at DRB length 2 (2 of 5 depths) with seed 551114
- Sampling 10 circuits at DRB length 4 (3 of 5 depths) with seed 551124
- Sampling 10 circuits at DRB length 8 (4 of 5 depths) with seed 551134
- Sampling 10 circuits at DRB length 16 (5 of 5 depths) with seed 551144


/home/jpmarceaux/Applications/pyGSTi/pygsti/layouts/evaltree.py:183: UserWarning: Created an evaluation tree that is inefficient: tree-size > 3 * #circuits !
This is likely due to the fact that the circuits being simulated do not have a
periodic structure. Consider using a different forward simulator (e.g. MapForwardSimulator).
  _warnings.warn(("Created an evaluation tree that is inefficient: tree-size > %d * #circuits !\n"


0.00014658295533900856


# [5] Sample the circuits 

In [131]:
num_samples=8000

# GST samples

def simulate_taking_data(data_template_filename):
    """Simulate taking 1-qubit data and filling the results into a template dataset.txt file"""
    datagen_model = noise_model
    pygsti.io.fill_in_empty_dataset_with_fake_data(datagen_model, data_template_filename, num_samples=num_samples)
    
pygsti.io.write_empty_protocol_data(exp_design, 'test_gst_dir', clobber_ok=True)

# -- fill in the dataset file in tutorial_files/test_gst_dir/data/dataset.txt --
simulate_taking_data("test_gst_dir/data/dataset.txt")  # REPLACE with actual data-taking

data = pygsti.io.load_data_from_dir('test_gst_dir')
dataset_gst = data.dataset

# [6] Setup the experiments

Each experiment consists of running MLE and the filter on GST circuits of fixed max length, then comparing on a fixed noise model with fixed circuit length

In [132]:
# make random circuits 
N_circs = 100
depth = 128
test_circuits = []
for n in range(N_circs):
    test_circuits.append(random_circuit(depth, MODEL_PACK.processor_spec()))

In [133]:
# Setup
%matplotlib

hilbert_dims = 2**len(MODEL_PACK.processor_spec().qubit_labels)
num_params = len(filter_model.to_vector())

max_itr = 5
itr_epsilon = 1e-4

Using matplotlib backend: Qt5Agg


In [134]:
# run linear GST
linear_design = pygsti.protocols.StandardGSTDesign(target_model, prep_fiducials, meas_fiducials,
                                            germs, [1])
linear_circuits = linear_design.all_circuits_needing_data
pygsti.io.write_empty_protocol_data(linear_design, 'test_gst_dir', clobber_ok=True)
simulate_taking_data("test_gst_dir/data/dataset.txt")  # REPLACE with actual data-taking
linear_data = pygsti.io.load_data_from_dir('test_gst_dir')
gst_protocol = pygsti.protocols.StandardGST('full TP,H+S')
results = gst_protocol.run(linear_data)
linear_hs_model = results.estimates['H+S'].models['final iteration estimate']

x_linear_hs = linear_hs_model.to_vector()

-- Std Practice:  Iter 1 of 2  (full TP) --: 
  --- Iterative GST: [##################################################] 100.0%  128 circuits ---
  Iterative GST Total Time: 0.8s
-- Std Practice:  Iter 2 of 2  (H+S) --: 
  --- Iterative GST: [##################################################] 100.0%  128 circuits ---
  Iterative GST Total Time: 1.1s


In [136]:
error_history = dict()
for i in range(0, len(maxLengths)):
    exdesign = pygsti.protocols.StandardGSTDesign(target_model, prep_fiducials, meas_fiducials,
                                                germs, maxLengths[0:i+1])
    gst_circuits = exdesign.all_circuits_needing_data
    
    
    # make this run's MLE 
    pygsti.io.write_empty_protocol_data(exdesign, 'test_gst_dir', clobber_ok=True)
    simulate_taking_data("test_gst_dir/data/dataset.txt")  # REPLACE with actual data-taking
    data = pygsti.io.load_data_from_dir('test_gst_dir')
    dataset_gst = data.dataset 
    gst_protocol = pygsti.protocols.StandardGST('full TP,H+S')
    results = gst_protocol.run(data)
    gst_hs_model = results.estimates['H+S'].models['final iteration estimate']
    gst_tp_model = results.estimates['full TP'].models['final iteration estimate']
    
    if i == 0:
        print(gst_hs_model.to_vector() - x_linear_hs)
    
    # setup filter
    P = 5*np.sqrt(rb_param)*np.eye(num_params)
    filter_model.from_vector(np.zeros(num_params))
    zero_kf = ExtendedKalmanFilter(filter_model.copy(), P)
    filter_model.from_vector(x_linear_hs)
    P = 2*np.sqrt(rb_param)*np.eye(num_params)
    seeded_kf = ExtendedKalmanFilter(filter_model.copy(), P)
    lkf = LinearKalmanFilter(np.zeros(num_params), P, hilbert_dims)

    # train on gst circuits
    for idx, circ in tqdm(enumerate(gst_circuits)): 
        zero_kf.update([circ], dataset_gst, jdict_gst, hdict_gst, 0.05*np.eye(2), max_itr, itr_epsilon)
        cvec = vector_from_outcomes(dataset_gst[circ].counts, hilbert_dims)
        pvec = vector_from_outcomes(REF_MODEL.probabilities(circ), hilbert_dims)
        lkf.update_filter(circ, cvec, pvec, jdict_gst[circ], 1e-2*np.eye(hilbert_dims))
        if circ not in linear_circuits:
            seeded_kf.update([circ], dataset_gst, jdict_gst, hdict_gst, 0.05*np.eye(2), max_itr, itr_epsilon)
            
    
    # regression model
    x_regression = regression_fit(gst_circuits, dataset_gst, REF_MODEL, jdict_gst)
    regression_model.from_vector(x_regression)
    
    # test performance on the training set 
    filter_model.from_vector(lkf.x)
    error_history[i] = compare_model_list(noise_model, [zero_kf.model, seeded_kf.model, gst_hs_model, gst_tp_model, regression_model, REF_MODEL, filter_model], test_circuits)

-- Std Practice:  Iter 1 of 2  (full TP) --: 
  --- Iterative GST: [##################################################] 100.0%  128 circuits ---
  Iterative GST Total Time: 0.6s
-- Std Practice:  Iter 2 of 2  (H+S) --: 
  --- Iterative GST: [##################################################] 100.0%  128 circuits ---
  Iterative GST Total Time: 2.0s
[-1.86671093e-03 -1.60833146e-03 -1.42557986e-06  5.15374256e-03
  5.15373725e-03  4.26762422e-12 -3.01877507e-04  3.20789820e-03
 -1.18538787e-06  5.15367900e-03  5.15377246e-03  3.90950579e-12
  6.03418164e-03  1.16805855e-04 -1.38082280e-03  1.17608138e-02
  3.35394423e-03  1.01716205e-02 -2.72792433e-04  2.20148589e-03
  6.35396116e-03 -2.65454294e-03 -1.05634250e-02  3.13607645e-03
  5.82239565e-03 -1.80470054e-03 -9.86128779e-04  2.08306275e-03
 -2.42091243e-05 -3.06850591e-03  2.53050942e-03  6.65644862e-03
 -9.55232539e-04 -2.42521230e-05  1.24802339e-02  5.94387887e-03]


128it [00:02, 55.76it/s]


Total number of circuits:  100


100it [00:19,  5.17it/s]


-- Std Practice:  Iter 1 of 2  (full TP) --: 
  --- Iterative GST: [##################################################] 100.0%  258 circuits ---
  Iterative GST Total Time: 1.2s
-- Std Practice:  Iter 2 of 2  (H+S) --: 
  --- Iterative GST: [##################################################] 100.0%  258 circuits ---
  Iterative GST Total Time: 3.4s


...

100it [00:19,  5.21it/s]


-- Std Practice:  Iter 1 of 2  (full TP) --: 
  --- Iterative GST: [##################################################] 100.0%  1570 circuits ---
  Iterative GST Total Time: 5.1s
-- Std Practice:  Iter 2 of 2  (H+S) --: 
  --- Iterative GST: [##################################################] 100.0%  1570 circuits ---
  Iterative GST Total Time: 6.0s


1570it [00:56, 27.57it/s]


Total number of circuits:  100


100it [00:19,  5.21it/s]


# [6] Plot results

In [137]:
error_history

{0: (array([0.0084458 , 0.00757642, 0.00617653, 0.01611301, 0.00932441,
         0.00470895, 0.00883451]),
  array([0.00457856, 0.0086678 , 0.00617063, 0.01454997, 0.00529241,
         0.00417952, 0.00488365])),
 1: (array([0.00593793, 0.00182852, 0.00318001, 0.00872625, 0.00603479,
         0.00470895, 0.00598263]),
  array([0.00396841, 0.00162945, 0.00270116, 0.00882736, 0.00398242,
         0.00417952, 0.0039741 ])),
 2: (array([0.00410514, 0.00116896, 0.00110657, 0.00348855, 0.00412324,
         0.00470895, 0.00411489]),
  array([0.00446261, 0.00089062, 0.00066974, 0.00331068, 0.00445223,
         0.00417952, 0.00445709])),
 3: (array([0.00470946, 0.0015023 , 0.00136497, 0.00217291, 0.0047121 ,
         0.00470895, 0.00470766]),
  array([0.00423097, 0.00141121, 0.00160608, 0.00251733, 0.00423141,
         0.00417952, 0.00423279])),
 4: (array([0.00222132, 0.00093231, 0.00057802, 0.00114811, 0.00391142,
         0.00470895, 0.00391086]),
  array([0.00219824, 0.0006723 , 0.00066312, 

In [139]:
plt.figure()
for i in range(len(error_history.keys())):
    pts1 = plt.scatter(i, error_history[i][0][0], c='blue', marker='v', label='EKF')
    if i != 0:
        pts2 = plt.scatter(i, error_history[i][0][1], c='green', marker='p', label='seeded EKF')
    pts3 = plt.scatter(i, error_history[i][0][2], c='red', marker='D', label='MLE H+S')
    pts4 = plt.scatter(i, error_history[i][0][3], c='magenta', marker='h', label='MLE TP')
    pts5 = plt.scatter(i, error_history[i][0][4], c='brown', marker='x', label='Regression')
    pts6 = plt.scatter(i, error_history[i][0][5], c='orange', marker='X', label='Ideal')
    pts7 = plt.scatter(i, error_history[i][0][6], c='pink', marker='*', label='LKF')
plt.legend([pts1, pts2, pts3, pts4, pts5, pts6, pts7], ['EKF', 'seeded EKF', 'MLE H+S', 'MLE TP', 'Regression', 'Ideal', 'LKF'])
plt.title('Filter performance vs MLE')
plt.xlabel('Max Germ Length (log)')
plt.ylabel('Prediction Error [TVD]')

Text(0, 0.5, 'Prediction Error [TVD]')

In [207]:
avg_eigenvalue_infidelity(noise_model, filter_model, GATES)

0.007146514537702616

In [208]:
avg_eigenvalue_infidelity(noise_model, gst_hs_model, GATES)

0.007201298314319704

In [209]:
avg_eigenvalue_infidelity(noise_model, gst_tp_model, GATES)

0.007203789666050693

In [210]:
avg_eigenvalue_infidelity(noise_model, REF_MODEL, GATES)

0.0035844924687810087